In [621]:
import pandas as pd
import csv
import clvehicles
import os
import os.path
from itertools import combinations
from datetime import datetime
import math
import numpy as np
import time

from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn import preprocessing


Enter in the make and model below. I created a list of some of the major cities in the US. The loop will create a unique csv for each set of city, make, and model.

In [351]:
#TODO make CLI capable of accepting cities with 2 words (i.e. New York, los, angeles)\
#TODO automate how to delete old csv files
cities = ['Phoenix', 'madison']
make = 'Acura'
model = 'MDX'

for i in cities:
    command = 'python clvehicles.py ' + i.lower() + ' ' + make.lower() + ' ' + model.lower()
    print('before os system command')
    os.system(command)
    print('after os system command')

    filename = i.lower() + model.lower() + '.csv'
    #Delete while loop? Likely redundant, the os.system(command) line doesn't continue until csv is generated
    while (not os.path.isfile('./' + filename)):
        print('not in folder yet')
        time.sleep(2)
    print('middle of while loop')


before os system command
after os system command
middle of while loop
before os system command
after os system command
middle of while loop


In [530]:
del df
del temp_df
del frames
frames = []
df = pd.DataFrame()

for i in cities:
    filename = i.lower() + model.lower() + '.csv'
    temp_df = pd.read_csv(filename)
    #df.append(temp_df, ignore_index = True)
    #display(temp_df)
    frames.append(temp_df)
    
df = pd.concat(frames, axis=0, ignore_index = True);

This command stops the df from truncating the url. It makes debugging by viewing the original posting much easier.

In [531]:
pd.set_option('display.max_colwidth', -1)

# Clean the Data
The next step is to determine how to eliminate duplicate rows. 

In [532]:
before_drop = len(df)
df.drop_duplicates()
after_drop = len(df)
display(str(before_drop - after_drop) +  ' duplicates were dropped')

'0 duplicates were dropped'

Because the URL is different on identical listings, it is not a good metric to determine duplicates. Furthermore, because we have aggregated all of the data by model and make, it is unreasonable to eliminate rows that have similar entries.
For example: https://phoenix.craigslist.org/wvl/cto/d/2007-acura-mdx-awd/6538180578.html and https://phoenix.craigslist.org/wvl/ctd/d/2007-acura-mdx/6537533572.html have the same model, make, year, price, location, and title status but are different listings. The only two feasible ways to eliminate duplicate rows is by comparing the images on two different pages, or comparing the times they were posted. Most duplicates are posted within a couple minutes of each other and have similar titles, so that will be the criteria for determining rows that should be deleted.
#### Update
Comparing times turned out to be error proned as expected. Did not catch all duplicates and actually removed unique listings. Onto picture parsing, I suppose

In [533]:
#TODO
#Fix these functions. Not all duplicates were posted at the same time. Sometimes listings have the same name
#and are different cars. Other times they have different names and are the same car. Currently working on this.
'''
def rowsAreSimilar(input_df, row1, row2):
    if input_df.name.iloc[row1] == input_df.name.iloc[row2]:
        return True
    if input_df.year.iloc[row1] == input_df.year.iloc[row2]:
        return True
    #Need more parameters
    return False


def eliminateRowsByTime(input_df, city):
    length = len(input_df)
    lst = np.arange(0,length,1)
    elims = []
    for combo in combinations(lst, 2):  #selects each combination (saves runtime rather than testing rows 0 vs 1 AND 1 vs 0.)
        #print(combo[1])
        time1  = (df[df['location'] == city].timestamp[combo[0]])
        time2  = (df[df['location'] == city].timestamp[combo[1]])
        dtime1 = datetime.strptime(time1, '%Y-%m-%d %H:%M')
        dtime2 = datetime.strptime(time2, '%Y-%m-%d %H:%M')
        diff = (dtime1 - dtime2)
        
        #This block is executed if there are two rows with almost identical timestamps. 
        #Now we compare the column values to see if they are the same car
        if (diff.total_seconds()/60) < 1.5: #if the difference of timestamps of the postings is <1.5 minutes
            if rowsAreSimilar(input_df, combo[0], combo[1]): #if the rows are similar
                elims.append(combo[1]) #has to be a duplicate posting
    print('elims are', elims)

c = df[df['location'] == 'chicago']
eliminateRowsByTime(c, 'chicago')
''';

In [534]:
#df[(df['location'] == 'madison') & (df['year'] < 2014)]

In [535]:
type(df['year'].iloc[50])

numpy.int64

Type cast all the values in the price column from 'str' to 'int'

In [536]:
#df['price'].replace('[\$, ]', '', regex=True).astype(float)
#df['price'] = df['price'].replace('$', '', regex=True)
df.price = df.price.replace('\$', '', regex=True).astype(float)

### Fix Odometer
Lazy people will put '90' miles when they actually mean '90000.' Cars displaying 0 miles will be overwritten as 'Nan'

In [537]:
for i in df[(df['milage'] < 500)].milage.index:
    if df['milage'].iloc[i] == 0:
        df.milage.iloc[i] = float('NaN')
    print("i is ", i, ' with the milage of ',df['milage'].iloc[i])
    df.milage.iloc[i] *= 1000    

('i is ', 3L, ' with the milage of ', 79.0)
('i is ', 31L, ' with the milage of ', 157.0)
('i is ', 94L, ' with the milage of ', nan)
('i is ', 95L, ' with the milage of ', nan)


In [538]:
df.isnull().sum()

name            0 
price           0 
year            0 
condition       41
milage          17
title status    0 
transmission    0 
drive           37
cylinders       33
fuel            0 
color           36
location        0 
timestamp       0 
url             0 
dtype: int64

## Prepare the Model
Drop the columns that are unnecessary to the model
All columns are: u'name', u'price', u'year', u'condition', u'milage', u'title status', u'transmission', u'drive', u'cylinders', u'fuel', u'color', u'location', u'timestamp', u'url

In [539]:
df = df.drop(['url', 'fuel', 'timestamp'], axis=1)

Cast the object types into floats so the model can accept them

In [540]:
df['condition'].unique()

array([' excellent', ' like new', nan, ' good', ' fair', ' new'],
      dtype=object)

Apply arbitrary variables to certain columns containing object types. We will use one-hot encoding for columns with values that have no correlation between the values, such as color.

In [541]:
condition_mapping = {' excellent': 5, ' new': 4, ' like new': 3, ' good': 2, ' fair': 1, np.nan : 0}
title_status_mapping = {' clean' : 1, ' rebuilt' : 0, np.nan : 0}
transmission_mapping = {' automatic' : 1, ' manual' : 0}
drive_mapping = {' 4wd' : 1, ' fwd' : 1, ' rwd' : 0, np.nan : .5}

In [542]:
df['condition'] = df['condition'].replace(condition_mapping)
df['title status'] = df['title status'].replace(title_status_mapping)
df['transmission'] = df['transmission'].replace(transmission_mapping)
df['drive'] = df['drive'].replace(drive_mapping)


In [543]:
df['cylinders'].unique()

array([' 6 cylinders', nan, ' 4 cylinders', ' other'], dtype=object)

In [544]:
#TODO eventually fix this and use one-hot encoding, but just leave it out of the pricing model for now
df = df.drop(['color'], axis = 1)
df = df.drop(['cylinders'], axis = 1)
df.head()

,name,price,year,condition,milage,title status,transmission,drive,location
0,2011 Acura MDX 62k miles,19700.0,2011,5,62000.0,1,1,1.0,phoenix
1,ACURA MDX 4X4,5600.0,2006,5,117000.0,0,1,1.0,phoenix
2,2008 Acura MDX W/Technology Package & Sunroof GORGEOUS!!!!,11500.0,2008,5,147000.0,1,1,1.0,phoenix
3,2010 Acura MDX Tech Package - 79K miles - Like new,18950.0,2010,3,79000.0,1,1,1.0,phoenix
4,2009 ACURA MDX,8800.0,2009,5,225000.0,1,1,1.0,phoenix


## Train the Model
we will use linear regression to train a pricing model based off of the information left in our dataframe

I'll fix this later, but for now, let's just drop any rows with null data
TODO: Parameterize the cities

In [547]:
df = df.dropna(axis = 0)

In [554]:
df[df['location'] == 'madison']['price']

87    5800.0 
88    14500.0
89    17995.0
90    38500.0
91    17999.0
92    30995.0
96    6995.0 
Name: price, dtype: float64

In [555]:
Y_madison = df[df['location'] == 'madison']['price']
Y_phoenix = df[df['location'] == 'phoenix']['price']
X = df.drop('price', axis = 1)
X = df.drop('name', axis = 1)

TODO: Parameterize the cities

In [609]:
lm = LinearRegression()
X_madison = X[X['location'] == 'madison']
X_phoenix = X[X['location'] == 'phoenix']
X_madison = X_madison.drop('location', axis = 1)
X_madison = X_madison.drop('price', axis = 1)
X_phoenix = X_phoenix.drop('location', axis = 1)
X_phoenix = X_phoenix.drop('price', axis = 1)

In [648]:
X_madison

,year,condition,milage,title status,transmission,drive
87,2005,2,232000.0,1,1,1.0
88,2009,5,91000.0,1,1,1.0
89,2012,5,97840.0,1,1,1.0
90,2016,3,12000.0,1,1,1.0
91,2012,0,62000.0,1,1,1.0
92,2016,3,68180.0,1,1,1.0
96,2007,5,206585.0,1,1,1.0


In [649]:
lm.fit(X_madison, Y_madison)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [650]:
X_madison.columns

Index([u'year', u'condition', u'milage', u'title status', u'transmission',
       u'drive'],
      dtype='object')

In [651]:
lm.predict([[2012, 3, 150000, 1, 1, 1]])

array([21104.72599549])

In [634]:
#scaler = StandardScaler()
#a = scaler.fit(X_madison)

In [635]:
#X_scaled = preprocessing.scale(X_madison)
#Y_scaled = preprocessing.scale(Y_madison)

In [640]:
#lm.fit(X_madison, Y_madison)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [647]:
#lm.predict([[2015, 5, 100000, 1, 1, 4]])

array([29246.77442769])